Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.5 MB/s eta 0:00:00


BİR GRAFTA RASTGELE YÜRÜME

Cirq Örneği: 1-boyutlu kuantum yürüyüşü

In [2]:
import cirq
import numpy as np

# Qubitler: pozisyon ve coin (1 qubit coin, 3 qubit pozisyon)
coin = cirq.LineQubit(0)
positions = [cirq.LineQubit(i) for i in range(1,4)]

circuit = cirq.Circuit()

# Başlangıç durumu: pozisyon 0, coin süperpozisyon
circuit.append(cirq.H(coin))
circuit.append(cirq.X(positions[0]))  # Başlangıç pozisyonu |1>

# 3 adımlık yürüyüş
for _ in range(3):
    # Coin: Hadamard
    circuit.append(cirq.H(coin))
    # Koşullu kayma
    for i in range(len(positions)):
        if i < len(positions)-1:
            circuit.append(cirq.CX(coin, positions[i+1]))
        if i > 0:
            circuit.append(cirq.CX(coin, positions[i-1]))

# Ölçüm
circuit.append([cirq.measure(q, key=f'p{i}') for i, q in enumerate(positions)])
circuit.append(cirq.measure(coin, key='coin'))

sim = cirq.Simulator()
result = sim.run(circuit, repetitions=10)
print(result)


coin=1110100010
p0=0101011110
p1=0000000000
p2=1010100001


Qiskit Örneği: 1-boyutlu kuantum yürüyüşü

In [4]:
from qiskit import QuantumCircuit, Aer, execute

qc = QuantumCircuit(4, 4)  # 1 coin + 3 pozisyon qubit

# Başlangıç durumu
qc.h(0)          # coin
qc.x(1)          # pozisyon |1>

# 3 adım
for _ in range(3):
    qc.h(0)  # coin
    qc.cx(0,1)  # koşullu kayma örneği
    qc.cx(0,2)
    qc.cx(0,3)

# Ölçüm
for i in range(4):
    qc.measure(i,i)

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1000).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP Örneği: 1-boyutlu kuantum yürüyüşü (matris temelli)

In [5]:
from qutip import *
import numpy as np

# Coin qubit + 3 pozisyon qubit
coin = basis(2,0)  # |0>
positions = [basis(2,0) for _ in range(3)]
state = tensor([coin]+positions)

# Hadamard operator (coin)
H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])

# Koşullu kayma operatorü (basitleştirilmiş)
shift = tensor([qeye(2)]*4)  # Basit placeholder

# 3 adım yürüyüş
for _ in range(3):
    H_op = tensor([H if i==0 else qeye(2) for i in range(4)])
    state = H_op * state
    state = shift * state

print(state)


Quantum object: dims=[[2, 2, 2, 2], [1]], shape=(16, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]


BİR GRAFTA KUANTUMSAL YÜRÜME

Cirq ile Kuantum Yürüme

In [6]:
import cirq
import numpy as np

# 3 pozisyon qubiti + 1 coin qubit
coin = cirq.LineQubit(0)
positions = [cirq.LineQubit(i) for i in range(1,4)]

circuit = cirq.Circuit()

# Başlangıç durumu: pozisyon 1 (|100>) ve coin süperpozisyon
circuit.append(cirq.H(coin))
circuit.append(cirq.X(positions[0]))

# 3 adım kuantum yürüme
for _ in range(3):
    circuit.append(cirq.H(coin))  # coin Hadamard
    for i in range(len(positions)-1):
        # Koşullu kayma (basitleştirilmiş)
        circuit.append(cirq.CX(coin, positions[i+1]))

# Ölçüm
circuit.append([cirq.measure(q, key=f'p{i}') for i, q in enumerate(positions)])
circuit.append(cirq.measure(coin, key='coin'))

sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)
print(result.histogram(key='p0'), result.histogram(key='p1'), result.histogram(key='p2'))


Counter({1: 1000}) Counter({0: 519, 1: 481}) Counter({0: 519, 1: 481})


Qiskit ile Kuantum Yürüme

In [7]:
from qiskit import QuantumCircuit, Aer, execute

qc = QuantumCircuit(4, 4)  # 1 coin + 3 pozisyon

# Başlangıç durumu
qc.h(0)      # coin süperpozisyon
qc.x(1)      # pozisyon 1

# 3 adım kuantum yürüme
for _ in range(3):
    qc.h(0)  # coin
    qc.cx(0,1)  # koşullu kayma (basitleştirilmiş)
    qc.cx(0,2)

# Ölçüm
for i in range(4):
    qc.measure(i,i)

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1000).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Kuantum Yürüme (Statevector)

In [8]:
from qutip import *
import numpy as np

# 3 pozisyon + 1 coin
coin = basis(2,0)
positions = [basis(2,0) for _ in range(3)]
state = tensor([coin]+positions)

# Hadamard (coin)
H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])

# Basitleştirilmiş koşullu kayma matrisi
shift = tensor([qeye(2)]*4)  # gerçek graf için detaylandırılabilir

# 3 adım yürüyüş
for _ in range(3):
    H_op = tensor([H if i==0 else qeye(2) for i in range(4)])
    state = H_op * state
    state = shift * state

# Ölçüm olasılıkları
probs = np.abs(state.full())**2
print(probs)


[[0.5]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0.5]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]]


KUANTUM YÜRÜMENİN MATEMATİKSEL TANIMI

Cirq ile Kuantum Para Atma (Coin Toss)

In [9]:
import cirq
import numpy as np

# Qubitler: coin ve pozisyon
coin = cirq.LineQubit(0)
position = cirq.LineQubit(1)

circuit = cirq.Circuit()

# Başlangıç: coin |0> ve pozisyon |0>
circuit.append(cirq.X(position))  # pozisyon başlangıç |1> de olabilir

# Coin Hadamard ile süperpozisyon
circuit.append(cirq.H(coin))

# Ölçüm
circuit.append(cirq.measure(coin, key='coin'))
circuit.append(cirq.measure(position, key='pos'))

sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)
print(result.histogram(key='coin'))
print(result.histogram(key='pos'))


Counter({1: 518, 0: 482})
Counter({1: 1000})


Qiskit ile Kuantum Para Atma

In [10]:
from qiskit import QuantumCircuit, Aer, execute

qc = QuantumCircuit(2, 2)  # 1 coin + 1 pozisyon

# Başlangıç
qc.x(1)   # pozisyon |1>
qc.h(0)   # coin süperpozisyon

# Ölçüm
qc.measure([0,1], [0,1])

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1000).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Matematiksel Durum ve Olasılık

In [11]:
from qutip import *
import numpy as np

# Başlangıç durumları
coin = basis(2,0)        # |0> coin
position = basis(2,1)    # |1> pozisyon
state = tensor(coin, position)

# Hadamard (coin)
H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])
H_op = tensor(H, qeye(2))
state = H_op * state

# Olasılık hesaplama
probs = np.abs(state.full())**2
print(probs)  # toplam olasılık = 1
print("Toplam olasılık:", np.sum(probs))


[[0. ]
 [0.5]
 [0. ]
 [0.5]]
Toplam olasılık: 0.9999999999999998


BİR ÇİZGİ ÜZERİNDE RASTGELE YÜRÜME

Cirq ile 1D Kuantum Yürüyüşü (Coin + Pozisyon)

In [12]:
import cirq
import numpy as np

n_steps = 5  # yürüyüş adımı
coin = cirq.LineQubit(0)
positions = [cirq.LineQubit(i+1) for i in range(2*n_steps+1)]  # -5..5 pozisyon

circuit = cirq.Circuit()

# Başlangıç: coin |0> ve pozisyon ortada |1>
circuit.append(cirq.X(positions[n_steps]))  # ortadaki pozisyon

# Adım fonksiyonu
def step(circuit, coin, positions):
    circuit.append(cirq.H(coin))  # coin süperpozisyon
    # Koşullu kaydırma
    for i in range(len(positions)-1):
        circuit.append(cirq.CNOT(coin, positions[i+1]))

# Adım sayısı kadar tekrar
for _ in range(n_steps):
    step(circuit, coin, positions)

# Ölçüm
circuit.append(cirq.measure(*positions, key='pos'))

sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)
print(result.histogram(key='pos'))


Counter({991: 500, 32: 500})


Qiskit ile 1D Kuantum Yürüyüşü

In [13]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

n_steps = 5
n_pos = 2*n_steps + 1

qc = QuantumCircuit(n_pos + 1, n_pos)  # 1 coin + pozisyon qubitleri
qc.x(n_steps+1)  # ortadaki pozisyon

# Adım fonksiyonu
def step(qc):
    qc.h(0)  # coin süperpozisyon
    for i in range(1, n_pos):
        qc.cx(0, i)

for _ in range(n_steps):
    step(qc)

# Ölçüm
qc.measure(range(1,n_pos+1), range(n_pos))

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1000).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Matematiksel Modelleme

In [14]:
from qutip import *
import numpy as np

n_steps = 5
n_pos = 2*n_steps + 1

# Başlangıç: coin |0>, pozisyon ortada
coin = basis(2,0)
positions = basis(n_pos, n_steps)
state = tensor(coin, positions)

# Hadamard coin
H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])
H_op = tensor(H, qeye(n_pos))

state = H_op * state  # coin süperpozisyon

# Ölçüm olasılıkları
probs = np.abs(state.full())**2
print("Olasılık dağılımı:", probs)
print("Toplam olasılık:", np.sum(probs))  # =1


Olasılık dağılımı: [[0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0.5]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0.5]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]]
Toplam olasılık: 0.9999999999999998


BİR ÇİZGİ ÜZERİNDE KUANTUMSAL YÜRÜME

Cirq ile 1D Kuantum Yürüyüşü (Coin + Pozisyon)

In [15]:
import cirq
import numpy as np

n_steps = 5
coin = cirq.LineQubit(0)
positions = [cirq.LineQubit(i+1) for i in range(2*n_steps+1)]  # pozisyon -n..+n

circuit = cirq.Circuit()

# Başlangıç: coin |L> ve pozisyon ortada
circuit.append(cirq.X(positions[n_steps]))

# Hadamard coin + kaydırma operatörü
def step(circuit, coin, positions):
    circuit.append(cirq.H(coin))  # coin süperpozisyon
    for i in range(len(positions)-1):
        circuit.append(cirq.CNOT(coin, positions[i+1]))  # L/R koşullu kaydırma

for _ in range(n_steps):
    step(circuit, coin, positions)

circuit.append(cirq.measure(*positions, key='pos'))
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)
print(result.histogram(key='pos'))


Counter({32: 539, 991: 461})


Qiskit ile 1D Kuantum Yürüyüşü (L/R Coin)

In [16]:
from qiskit import QuantumCircuit, Aer, execute

n_steps = 5
n_pos = 2*n_steps + 1

qc = QuantumCircuit(n_pos + 1, n_pos)  # 1 coin + pozisyon qubitleri
qc.x(n_steps+1)  # ortadaki pozisyon

def step(qc):
    qc.h(0)  # coin süperpozisyon
    for i in range(1, n_pos):
        qc.cx(0, i)  # koşullu kaydırma

for _ in range(n_steps):
    step(qc)

qc.measure(range(1,n_pos+1), range(n_pos))
backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1000).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Matematiksel Modelleme (L/R Coin)

In [17]:
from qutip import *
import numpy as np

n_steps = 5
n_pos = 2*n_steps + 1

# Coin |L> = |0>, |R> = |1>, pozisyon ortada
coin = basis(2,0)
positions = basis(n_pos, n_steps)
state = tensor(coin, positions)

# Hadamard coin
H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])
H_op = tensor(H, qeye(n_pos))
state = H_op * state

# Shift operatörü
shift_left = sum([tensor(basis(2,0)*basis(2,0).dag(), basis(n_pos, max(i-1,0))*basis(n_pos,i).dag()) for i in range(n_pos)])
shift_right = sum([tensor(basis(2,1)*basis(2,1).dag(), basis(n_pos, min(i+1,n_pos-1))*basis(n_pos,i).dag()) for i in range(n_pos)])
shift = shift_left + shift_right

state = shift * state
probs = np.abs(state.full())**2
print("Olasılık dağılımı:", probs.flatten())
print("Toplam olasılık:", np.sum(probs))


Olasılık dağılımı: [0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5
 0.  0.  0.  0. ]
Toplam olasılık: 0.9999999999999998


YÖNLÜ OLMAYAN KUANTUM YÜRÜME DAVRANIŞI

Cirq ile Yönsüz Kuantum Yürüyüşü

In [18]:
import cirq
import numpy as np

n_steps = 4
coin = cirq.LineQubit(0)
positions = [cirq.LineQubit(i+1) for i in range(5)]  # 5 pozisyonlu yönsüz graf

circuit = cirq.Circuit()
circuit.append(cirq.X(positions[2]))  # başlangıç ortada

def step(circuit, coin, positions):
    circuit.append(cirq.H(coin))
    for i in range(len(positions)):
        if i > 0:
            circuit.append(cirq.CNOT(coin, positions[i-1]))  # sola kaydır
        if i < len(positions)-1:
            circuit.append(cirq.CNOT(coin, positions[i+1]))  # sağa kaydır

for _ in range(n_steps):
    step(circuit, coin, positions)

circuit.append(cirq.measure(*positions, key='pos'))
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)
print(result.histogram(key='pos'))


Counter({21: 1000})


Qiskit ile Yönsüz Kuantum Yürüyüşü

In [19]:
from qiskit import QuantumCircuit, Aer, execute

n_pos = 5
n_steps = 4
qc = QuantumCircuit(n_pos + 1, n_pos)  # coin + pozisyon qubitleri
qc.x(3)  # ortadaki pozisyon

def step(qc):
    qc.h(0)  # coin süperpozisyon
    for i in range(1, n_pos+1):
        if i > 1:
            qc.cx(0, i-1)  # sola kaydır
        if i < n_pos:
            qc.cx(0, i)  # sağa kaydır

for _ in range(n_steps):
    step(qc)

qc.measure(range(1, n_pos+1), range(n_pos))
backend = Aer.get_backend('qasm_simulator')
counts = execute(qc, backend, shots=1000).result().get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Yönsüz Kuantum Yürüyüşü

In [20]:
from qutip import *
import numpy as np

n_pos = 5
n_steps = 4

# Başlangıç: coin |0> ve pozisyon ortada
coin = basis(2,0)
positions = basis(n_pos, n_pos//2)
state = tensor(coin, positions)

H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])
H_op = tensor(H, qeye(n_pos))

shift_left = sum([tensor(basis(2,0)*basis(2,0).dag(), basis(n_pos, max(i-1,0))*basis(n_pos,i).dag()) for i in range(n_pos)])
shift_right = sum([tensor(basis(2,1)*basis(2,1).dag(), basis(n_pos, min(i+1,n_pos-1))*basis(n_pos,i).dag()) for i in range(n_pos)])
shift = shift_left + shift_right

for _ in range(n_steps):
    state = H_op * state
    state = shift * state

probs = np.abs(state.full())**2
print("Olasılık dağılımı:", probs.flatten())
print("Toplam olasılık:", np.sum(probs))


Olasılık dağılımı: [1.     0.     0.0625 0.0625 0.     0.     0.0625 0.0625 0.     0.25  ]
Toplam olasılık: 1.4999999999999991


TERSLENEBİLEN VE TERSLENEMEYEN GRAFLAR

Cirq ile Terslenebilir Kuantum Yürüyüşü

In [21]:
import cirq
import numpy as np
import matplotlib.pyplot as plt

# Qubitler: coin + pozisyonlar
n_pos = 5
coin = cirq.LineQubit(0)
positions = [cirq.LineQubit(i+1) for i in range(n_pos)]

circuit = cirq.Circuit()
circuit.append(cirq.X(positions[2]))  # Başlangıç ortada

def step(circuit, coin, positions):
    circuit.append(cirq.H(coin))
    for i in range(len(positions)):
        if i > 0:
            circuit.append(cirq.CNOT(coin, positions[i-1]))
        if i < len(positions)-1:
            circuit.append(cirq.CNOT(coin, positions[i+1]))

# 4 adım yürüyüş
for _ in range(4):
    step(circuit, coin, positions)

circuit.append(cirq.measure(*positions, key='pos'))
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)
counts = result.histogram(key='pos')
print("Olasılık dağılımı:", counts)


Olasılık dağılımı: Counter({21: 1000})


Qiskit ile Terslenebilir Kuantum Yürüyüşü

In [22]:
from qiskit import QuantumCircuit, Aer, execute

n_pos = 5
qc = QuantumCircuit(n_pos + 1, n_pos)  # coin + pozisyon
qc.x(3)  # başlangıç ortada

def step(qc):
    qc.h(0)  # coin
    for i in range(1, n_pos+1):
        if i > 1:
            qc.cx(0, i-1)  # sola kaydır
        if i < n_pos:
            qc.cx(0, i)  # sağa kaydır

for _ in range(4):
    step(qc)

qc.measure(range(1, n_pos+1), range(n_pos))
backend = Aer.get_backend('qasm_simulator')
counts = execute(qc, backend, shots=1000).result().get_counts()
print("Olasılık dağılımı:", counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Terslenebilir Kuantum Yürüyüşü

In [23]:
from qutip import *
import numpy as np

n_pos = 5
n_steps = 4

# Başlangıç: coin |0> ve pozisyon ortada
coin = basis(2,0)
positions = basis(n_pos, n_pos//2)
state = tensor(coin, positions)

H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])
H_op = tensor(H, qeye(n_pos))

# Kaydırma operatörleri
shift_left = sum([tensor(basis(2,0)*basis(2,0).dag(), basis(n_pos, max(i-1,0))*basis(n_pos,i).dag()) for i in range(n_pos)])
shift_right = sum([tensor(basis(2,1)*basis(2,1).dag(), basis(n_pos, min(i+1,n_pos-1))*basis(n_pos,i).dag()) for i in range(n_pos)])
shift = shift_left + shift_right

for _ in range(n_steps):
    state = H_op * state
    state = shift * state

probs = np.abs(state.full())**2
print("Olasılık dağılımı:", probs.flatten())
print("Toplam olasılık:", np.sum(probs))


Olasılık dağılımı: [1.     0.     0.0625 0.0625 0.     0.     0.0625 0.0625 0.     0.25  ]
Toplam olasılık: 1.4999999999999991
